<p> Data extracted from the NBA API (See Apis: NBA) is missing (Michael Jordan's stats are incomplete, several seasons are missing), we'll get this data from ESPN).

We tried to use Beautiful Soup</p>

In [ ]:
pip install selenium pandas webdriver-manager

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
pip install --upgrade selenium

## Regular Season data

In [2]:

# Function to initialize the WebDriver
def init_driver():
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")  # Runs in the background
        driver = webdriver.Chrome(options=options)
        return driver
    except Exception as e:
        print(f"Error initializing WebDriver: {e}")
        return None

# Open the website
url = "https://www.espn.com/nba/player/stats/_/id/1035/type/nba/seasontype/2"

# Try initializing the WebDriver
driver = init_driver()
if driver is None:
    print("Failed to initialize the WebDriver. Exiting script.")
else:
    try:
        driver.get(url)

        # Wait for the table to load
        wait = WebDriverWait(driver, 10)

        # Find the table based on its title
        table_title = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='Table__Title' and text()='Regular Season Totals']")))

        # Get the parent table of the title
        table = table_title.find_element(By.XPATH, "ancestor::div[contains(@class, 'ResponsiveTable')]")

        # Extract headers (including 'SEASON' and 'TEAM')
        headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]
        
        # Extract rows
        data = []
        rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # Skip header row

        # Ensure each row has the correct number of columns
        for row in rows:
            cols = [td.text.strip() for td in row.find_elements(By.TAG_NAME, "td")]

            # Skip empty rows or rows with summary data like "Career"
            if not cols or cols[0] == 'Career':
                continue

            # Check if the number of columns matches the header length
            if len(cols) != len(headers):
                print(f"Row length mismatch: {len(cols)} columns found, but {len(headers)} expected. Padding the row.")
                # If the row has fewer columns than expected, pad with None
                if len(cols) < len(headers):
                    cols.extend([None] * (len(headers) - len(cols)))  # Pad missing columns with None

            data.append(cols)

        # Convert to DataFrame
        df = pd.DataFrame(data, columns=headers)

        # Clean up: Remove rows with too many missing values or empty rows
        df.dropna(how='all', inplace=True)  # Drop rows where all values are NaN
        df.reset_index(drop=True, inplace=True)  # Reset index after dropping rows

        # Show the cleaned DataFrame
        print(df)

    except Exception as e:
        print(f"Error during scraping: {e}")

    finally:
        # Close the WebDriver
        driver.quit()

Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding t

In [3]:
df

,SEASON,TEAM,FG,FG%,3PT,3P%,FT,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
0,1985,CHI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1986,CHI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1987,CHI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,1988,CHI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,1989,CHI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,1990,CHI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,1991,CHI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,1992,CHI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,1993,CHI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,1994-95,CHI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [4]:
df1=df.iloc[:14,:2]
df1

,SEASON,TEAM
0,1985,CHI
1,1986,CHI
2,1987,CHI
3,1988,CHI
4,1989,CHI
5,1990,CHI
6,1991,CHI
7,1992,CHI
8,1993,CHI
9,1994-95,CHI


In [5]:
df2=df.iloc[15:,]
df2.columns

Index(['SEASON', 'TEAM', 'FG', 'FG%', '3PT', '3P%', 'FT', 'FT%', 'OR', 'DR',
       'REB', 'AST', 'BLK', 'STL', 'PF', 'TO', 'PTS'],
      dtype='object')

In [7]:
df2.columns= ['FG', 'FG%', '3PT', '3P%', 'FT', 'FT%', 'OR', 'DR', 'REB', 'AST', 'BLK', 'STL', 'PF', 'TO', 'PTS', 'SEASON', 'TEAM']
df2.drop(["SEASON",'TEAM'],axis=1,inplace=True)

c:\Users\Abraham\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df3 = pd.concat([df1, df2], axis=1)

In [9]:
df3

,SEASON,TEAM,FG,FG%,3PT,3P%,FT,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
0,1985,CHI,837-1625,51.5,9-52,17.3,630-746,84.4,167,367,534,481,69,196,285,291,2313
1,1986,CHI,150-328,45.7,3-18,16.7,105-125,84.0,23,41,64,53,21,37,46,45,408
2,1987,CHI,1098-2279,48.2,12-66,18.2,833-972,85.7,166,264,430,377,125,236,237,272,3041
3,1988,CHI,1069-1998,53.5,7-53,13.2,723-860,84.1,139,310,449,485,131,259,270,252,2868
4,1989,CHI,966-1795,53.8,27-98,27.6,674-793,85.0,149,503,652,650,65,234,247,290,2633
5,1990,CHI,1034-1964,52.6,92-245,37.6,593-699,84.8,143,422,565,519,54,227,241,247,2753
6,1991,CHI,990-1837,53.9,29-93,31.2,571-671,85.1,118,374,492,453,83,223,229,202,2580
7,1992,CHI,943-1818,51.9,27-100,27.0,491-590,83.2,91,420,511,489,75,182,201,200,2404
8,1993,CHI,992-2003,49.5,81-230,35.2,476-569,83.7,135,387,522,428,61,221,188,207,2541
9,1994-95,CHI,166-404,41.1,16-32,50.0,109-136,80.1,25,92,117,90,13,30,47,35,457


In [11]:
df3.loc[14, "SEASON"]="2002-03"
df3.loc[14,"TEAM"]= "WSH"
df3.drop(15,axis=0)

,SEASON,TEAM,FG,FG%,3PT,3P%,FT,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
0,1985,CHI,837-1625,51.5,9-52,17.3,630-746,84.4,167,367,534,481,69,196,285,291,2313
1,1986,CHI,150-328,45.7,3-18,16.7,105-125,84.0,23,41,64,53,21,37,46,45,408
2,1987,CHI,1098-2279,48.2,12-66,18.2,833-972,85.7,166,264,430,377,125,236,237,272,3041
3,1988,CHI,1069-1998,53.5,7-53,13.2,723-860,84.1,139,310,449,485,131,259,270,252,2868
4,1989,CHI,966-1795,53.8,27-98,27.6,674-793,85.0,149,503,652,650,65,234,247,290,2633
5,1990,CHI,1034-1964,52.6,92-245,37.6,593-699,84.8,143,422,565,519,54,227,241,247,2753
6,1991,CHI,990-1837,53.9,29-93,31.2,571-671,85.1,118,374,492,453,83,223,229,202,2580
7,1992,CHI,943-1818,51.9,27-100,27.0,491-590,83.2,91,420,511,489,75,182,201,200,2404
8,1993,CHI,992-2003,49.5,81-230,35.2,476-569,83.7,135,387,522,428,61,221,188,207,2541
9,1994-95,CHI,166-404,41.1,16-32,50.0,109-136,80.1,25,92,117,90,13,30,47,35,457


In [12]:
df3 = df3.iloc[:-1]

In [14]:
df3.to_csv("jordan_regular")

## Playoffs Data

In [16]:

# Function to initialize the WebDriver
def init_driver():
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")  # Runs in the background
        driver = webdriver.Chrome(options=options)
        return driver
    except Exception as e:
        print(f"Error initializing WebDriver: {e}")
        return None

# Open the website
url = "https://www.espn.com/nba/player/stats/_/id/1035/type/nba/seasontype/3"

# Try initializing the WebDriver
driver = init_driver()
if driver is None:
    print("Failed to initialize the WebDriver. Exiting script.")
else:
    try:
        driver.get(url)

        # Wait for the table to load
        wait = WebDriverWait(driver, 10)

        # Find the table based on its title
        table_title = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='Table__Title' and text()='Postseason Totals']")))

        # Get the parent table of the title
        table = table_title.find_element(By.XPATH, "ancestor::div[contains(@class, 'ResponsiveTable')]")

        # Extract headers (including 'SEASON' and 'TEAM')
        headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]
        
        # Extract rows
        data = []
        rows = table.find_elements(By.TAG_NAME, "tr")[1:]  # Skip header row

        # Ensure each row has the correct number of columns
        for row in rows:
            cols = [td.text.strip() for td in row.find_elements(By.TAG_NAME, "td")]

            # Skip empty rows or rows with summary data like "Career"
            if not cols or cols[0] == 'Career':
                continue

            # Check if the number of columns matches the header length
            if len(cols) != len(headers):
                print(f"Row length mismatch: {len(cols)} columns found, but {len(headers)} expected. Padding the row.")
                # If the row has fewer columns than expected, pad with None
                if len(cols) < len(headers):
                    cols.extend([None] * (len(headers) - len(cols)))  # Pad missing columns with None

            data.append(cols)

        # Convert to DataFrame
        df_playoff = pd.DataFrame(data, columns=headers)

        # Clean up: Remove rows with too many missing values or empty rows
        df_playoff.dropna(how='all', inplace=True)  # Drop rows where all values are NaN
        df_playoff.reset_index(drop=True, inplace=True)  # Reset index after dropping rows

        # Show the cleaned DataFrame
        print(df)

    except Exception as e:
        print(f"Error during scraping: {e}")

    finally:
        # Close the WebDriver
        driver.quit()

Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 2 columns found, but 17 expected. Padding the row.
Row length mismatch: 15 columns found, but 17 expected. Padding 

In [44]:
 df_playoff1= df_playoff.iloc[:13]


In [45]:
df_playoff1= df_playoff1[["SEASON","TEAM"]]
df_playoff1

,SEASON,TEAM
0,1985,CHI
1,1986,CHI
2,1987,CHI
3,1988,CHI
4,1989,CHI
5,1990,CHI
6,1991,CHI
7,1992,CHI
8,1993,CHI
9,1994-95,CHI


In [46]:
df_playoff2=df_playoff.iloc[13:]
df_playoff2

,SEASON,TEAM,FG,FG%,3PT,3P%,FT,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
13,34-78,43.6,1-8,12.5,48-58,82.8,7,16,23,34,4,11,15,15,117,None,None
14,48-95,50.5,1-1,100.0,34-39,87.2,5,14,19,17,4,7,13,14,131,None,None
15,35-84,41.7,2-5,40.0,35-39,89.7,7,14,21,18,7,6,11,8,107,None,None
16,138-260,53.1,1-3,33.3,86-99,86.9,23,48,71,47,11,24,38,39,363,None,None
17,199-390,51.0,10-35,28.6,183-229,79.9,26,93,119,130,13,42,65,68,591,None,None
18,219-426,51.4,16-50,32.0,133-159,83.6,24,91,115,109,14,45,54,56,587,None,None
19,197-376,52.4,10-26,38.5,125-148,84.5,18,90,108,142,23,40,53,43,529,None,None
20,290-581,49.9,17-44,38.6,162-189,85.7,37,100,137,127,16,44,62,81,759,None,None
21,251-528,47.5,28-72,38.9,136-169,80.5,32,96,128,114,17,39,58,45,666,None,None
22,120-248,48.4,11-30,36.7,64-79,81.0,20,45,65,45,14,23,30,41,315,None,None


In [47]:
df_playoff2.columns= ['FG', 'FG%', '3PT', '3P%', 'FT', 'FT%', 'OR', 'DR', 'REB', 'AST', 'BLK', 'STL', 'PF', 'TO', 'PTS', 'SEASON', 'TEAM']
df_playoff2.drop(["SEASON",'TEAM'],axis=1,inplace=True)

c:\Users\Abraham\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [51]:
df_playoff1

,SEASON,TEAM
0,1985,CHI
1,1986,CHI
2,1987,CHI
3,1988,CHI
4,1989,CHI
5,1990,CHI
6,1991,CHI
7,1992,CHI
8,1993,CHI
9,1994-95,CHI


In [52]:
df_playoff1 =df_playoff1.reset_index(drop=True)
df_playoff2= df_playoff2.reset_index(drop=True)
df_playoff_full = pd.concat([df_playoff1, df_playoff2], axis=1)

In [53]:
df_playoff_full

,SEASON,TEAM,FG,FG%,3PT,3P%,FT,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
0,1985,CHI,34-78,43.6,1-8,12.5,48-58,82.8,7,16,23,34,4,11,15,15,117
1,1986,CHI,48-95,50.5,1-1,100.0,34-39,87.2,5,14,19,17,4,7,13,14,131
2,1987,CHI,35-84,41.7,2-5,40.0,35-39,89.7,7,14,21,18,7,6,11,8,107
3,1988,CHI,138-260,53.1,1-3,33.3,86-99,86.9,23,48,71,47,11,24,38,39,363
4,1989,CHI,199-390,51.0,10-35,28.6,183-229,79.9,26,93,119,130,13,42,65,68,591
5,1990,CHI,219-426,51.4,16-50,32.0,133-159,83.6,24,91,115,109,14,45,54,56,587
6,1991,CHI,197-376,52.4,10-26,38.5,125-148,84.5,18,90,108,142,23,40,53,43,529
7,1992,CHI,290-581,49.9,17-44,38.6,162-189,85.7,37,100,137,127,16,44,62,81,759
8,1993,CHI,251-528,47.5,28-72,38.9,136-169,80.5,32,96,128,114,17,39,58,45,666
9,1994-95,CHI,120-248,48.4,11-30,36.7,64-79,81.0,20,45,65,45,14,23,30,41,315


In [54]:
df_playoff_full= df_playoff_full.iloc[:-1]
df_playoff_full

,SEASON,TEAM,FG,FG%,3PT,3P%,FT,FT%,OR,DR,REB,AST,BLK,STL,PF,TO,PTS
0,1985,CHI,34-78,43.6,1-8,12.5,48-58,82.8,7,16,23,34,4,11,15,15,117
1,1986,CHI,48-95,50.5,1-1,100.0,34-39,87.2,5,14,19,17,4,7,13,14,131
2,1987,CHI,35-84,41.7,2-5,40.0,35-39,89.7,7,14,21,18,7,6,11,8,107
3,1988,CHI,138-260,53.1,1-3,33.3,86-99,86.9,23,48,71,47,11,24,38,39,363
4,1989,CHI,199-390,51.0,10-35,28.6,183-229,79.9,26,93,119,130,13,42,65,68,591
5,1990,CHI,219-426,51.4,16-50,32.0,133-159,83.6,24,91,115,109,14,45,54,56,587
6,1991,CHI,197-376,52.4,10-26,38.5,125-148,84.5,18,90,108,142,23,40,53,43,529
7,1992,CHI,290-581,49.9,17-44,38.6,162-189,85.7,37,100,137,127,16,44,62,81,759
8,1993,CHI,251-528,47.5,28-72,38.9,136-169,80.5,32,96,128,114,17,39,58,45,666
9,1994-95,CHI,120-248,48.4,11-30,36.7,64-79,81.0,20,45,65,45,14,23,30,41,315


In [55]:
df_playoff_full.to_csv("jordan_playoffs")

## 